<h1>Final assignment: Voorspellen met Monte Carlo</h1>
<h2>Door: Dennis Besselsen V2A</h2>

Als eerst ga ik de tabel met kansen (winst/ gelijk/ verlies tijdens thuiswedstrijden) vullen aan de hand van de data op canvas.

Dit doe ik d.m.v. van een dictionary.

Elke team heeft 1 key in de dict. Elke value is een lijst van kansen. De order van values in deze lijsten is gelijk als die van de keys, ofwel:

 - index[0]=Ajax
 - index[1]=Feyenoord
 - index[2]=PSV
 - index[3]=FC Utrecht
 - index[4]=Willem II

Teams kunnen niet tegen zichzelf spelen, dus deze plekken in de lijsten worden gevuld met None.

In [2]:
chance_dict = { 'Ajax':         [None,      '65/17/18', '54/21/25', '74/14/12', '78/13/9'   ],
                'Feyenoord':    ['30/21/49',None,       '37/24/39', '51/22/27', '60/21/19'  ],
                'PSV':          ['39/22/39','54/22/24', None,       '62/20/18', '62/22/16'  ],
                'FC Utrecht':   ['25/14/61','37/23/40', '29/24/47', None,       '52/23/25'  ],
                'Willem II':    ['17/18/65','20/26/54', '23/24/53', '37/25/38', None        ]}
                

In [3]:
class MidSquareGen(object):   # Algemene opzet voor een RNG object: init, en next. Optioneel (re)set seed.
    def __init__(self, seed):
        self.seed = seed
        
    def reset(self, seed):
        self.seed = seed
        
    def next(self):
        self.seed = MidSquareGen.select_middle(self.seed**2, 4) # Bereken volgende seed (en bewaar die)
        return self.seed / 10000.0                 # Bereken waarde uit seed (kan de geschaalde seed zijn, of een complexere functie)

    @staticmethod
    def select_middle(number, positions):       # Statische helperfunctie
        left = True
        while len(str(number)) > positions:
            if left: 
                number = int(str(number)[1:])
                left = False
            else:
                number = int(str(number)[:-1])
                left = True
        return number
    
rng_ms = MidSquareGen(5566778885464564563567324524978654423)

In [4]:
#Source: https://github.com/yinengy/Mersenne-Twister-in-Python

class Twister():
    def __init__(self, c_seed=0):
        # MT19937
        (self.w, self.n, self.m, self.r) = (32, 624, 397, 31)
        self.a = 0x9908B0DF
        (self.u, self.d) = (11, 0xFFFFFFFF)
        (self.s, self.b) = (7, 0x9D2C5680)
        (self.t, self.c) = (15, 0xEFC60000)
        self.l = 18
        self.f = 1812433253
        # make a arry to store the state of the generator
        self.MT = [0 for i in range(self.n)]
        self.index = self.n+1
        self.lower_mask = 0x7FFFFFFF
        self.upper_mask = 0x80000000
        # inital the seed
        self.c_seed = c_seed
        self.seed(c_seed)

    def seed(self, num):
        """initialize the generator from a seed"""
        self.MT[0] = num
        self.index = self.n
        for i in range(1, self.n):
            temp = self.f * (self.MT[i-1] ^ (self.MT[i-1] >> (self.w-2))) + i
            self.MT[i] = temp & 0xffffffff

    def twist(self):
        """ Generate the next n values from the series x_i"""
        for i in range(0, self.n):
            x = (self.MT[i] & self.upper_mask) + \
                (self.MT[(i+1) % self.n] & self.lower_mask)
            xA = x >> 1
            if (x % 2) != 0:
                xA = xA ^ self.a
            self.MT[i] = self.MT[(i + self.m) % self.n] ^ xA
        self.index = 0

    def extract_number(self):
        """ Extract a tempered value based on MT[index]
            calling twist() every n numbers
        """
        if self.index >= self.n:
            self.twist()

        y = self.MT[self.index]
        y = y ^ ((y >> self.u) & self.d)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        y = y ^ (y >> self.l)

        self.index += 1
        return y & 0xffffffff

    def random(self):
        """ return uniform ditribution in [0,1) """
        # a = (self.extract_number() / 10**8) % 1
        # return float('%.08f' % a)
        return self.extract_number() / 4294967296  # which is 2**w

    def randint(self, a, b):
        """ return random int in [a,b) """
        n = self.random()
        return int(n/(1/(b-a)) + a)

#Seed '1337420' heb ik handmatig ingevuld 
rng_mt = Twister(1337420)

Ik wil de wedstrijden als volgt simuleren

Team A(thuis) tegen Team B heeft als kansen: 33/33/34

Vervolgens laat ik een number generator een getal tussen te 0 en 1 maken.





In [32]:

def WDL_sim(chance_dict,generator):
    
    # Dictionary waar de score wordt bijgehouden
    score_dict = { 'Ajax':0,
                'Feyenoord':0,
                'PSV':0,
                'FC Utrecht':0,
                'Willem II':0}

    # Dictionary wat gebruikt wordt voor het verdelen van punten aan de 'away' teams
    away_dict = { 0:'Ajax',
                1:'Feyenoord',
                2:'PSV',
                3:'FC Utrecht',
                4:'Willem II'}


    for k,values in chance_dict.items():
        for value in values:
            if value != None:

                # split de value op elke  '/' , en deel ze waardes door 100 zodat het komma getallen worden tussen 0 en 1
                float_list = [(float(s)/100) for s in value.split('/')]

                # win_perc is de eerste value in de list, loss_perc is de eerste 2 values in de list bij elkaar opgeteld.
                win_perc = float_list[0]
                loss_perc = float_list[1]+float_list[0]
                if generator == 'mt':
                    gen_chance = rng_mt.random()
                elif generator == 'ms':                   
                    gen_chance = rng_ms.next()
                else:
                    print("Er is geen geldige PRNG aangegeven. ABORTING")
                    
                if gen_chance <= win_perc:
                    score_dict[k] += 3                              # home team points +3

                elif gen_chance > win_perc and gen_chance < loss_perc:
                    score_dict[k] += 1                              # home team points +1
                    score_dict[away_dict[values.index(value)]]+=1   # away team points +1
                    
                    
                else:  # alle andere opties zijn geelimineerd dus ik kan hier else gebruiken

                    score_dict[away_dict[values.index(value)]]+=3   # away team points +3
    
    # Hier worden de teams op score gesorteerd.  Het result is een list van teams op volgorde van position in het kampioenschap
    sorted_positions = sorted(score_dict, key=score_dict.get, reverse=True)
                 
    return sorted_positions
                
                

In [33]:
def sim_spam(iterations, generator):
    #dict waar ieder team 
    placement_dict = {  'Ajax':         [0,0,0,0,0],
                        'Feyenoord':    [0,0,0,0,0],
                        'PSV':          [0,0,0,0,0],
                        'FC Utrecht':   [0,0,0,0,0],
                        'Willem II':    [0,0,0,0,0]}
                
    for _ in range(iterations):
        position_list = WDL_sim(chance_dict,generator)
        for idx, val in enumerate(position_list):
            placement_dict[val][idx] +=1
    for k,values in placement_dict.items():
        templist=[]
        for value in values:
            templist.append(round(value / iterations*100,2))
        placement_dict[k] = templist
    return placement_dict

In [34]:
abv = sim_spam(10000, 'mt')
abv


{'Ajax': [61.65, 24.14, 9.59, 3.78, 0.84],
 'Feyenoord': [12.08, 25.9, 31.34, 21.1, 9.58],
 'PSV': [21.58, 35.43, 26.27, 12.44, 4.28],
 'FC Utrecht': [3.76, 10.46, 21.7, 37.3, 26.78],
 'Willem II': [0.93, 4.07, 11.1, 25.38, 58.52]}